**Acoustic Feature Extraction from Spotify API using spotipy**

### Install spotipy

In [ ]:
!pip install spotipy

     |████████████████████████████████| 153kB 8.0MB/s 
     |████████████████████████████████| 61kB 4.4MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


### Import Libraries 

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import pickle
import calendar
import datetime as dt
import csv
import os 
import pandas as pd
import numpy as np
import json

### Pass your spotify client id and spotify client secret key

In [ ]:
SPOTIPY_CLIENT_ID=''
SPOTIPY_CLIENT_SECRET=''
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Function to Get Track ID using track and artist name

In [ ]:
def get_track_id(track,artist):
    query = f"track:{track} artist:{artist}"
    track_data = sp.search(q=query, type='track', limit=1)
    items = track_data['tracks']['items']
    if not items:
        #print(f'Track Not Found for artist:{artist_name} track:{track_name}')
        return None
    else:
        return items[0]['id']

In [ ]:
track_id = get_track_id('The Red Sea', 'Peter Thomas Sound Orchester')
print(track_id)

5s0fFpGFpvEVf9GM3S2TsE


### Function to get acoustic feature of a track using track_id

In [ ]:
# "danceability": 0.808,
# "energy": 0.626,
# "key": 7,
# "loudness": -12.733,
# "mode": 1,
# "speechiness": 0.168,
# "acousticness": 0.00187,
# "instrumentalness": 0.159,
# "liveness": 0.376,
# "valence": 0.369,
# "tempo": 123.99,

In [ ]:
def get_features_by_track_ids(track_ids):
    feature_data = sp.audio_features(tracks=track_ids)
    list_features=[]
    for data in feature_data:
        if(data is not None):
            features= []
            features.append(data['danceability'])
            features.append(data['energy'])
            features.append(data['key'])
            features.append(data['loudness'])
            features.append(data['mode'])
            features.append(data['speechiness'])
            features.append(data['acousticness'])
            features.append(data['instrumentalness'])
            features.append(data['liveness'])
            features.append(data['valence'])
            features.append(data['tempo'])
        else:
            features=None
        list_features.append(features)
    return list_features

### Setting Folder for reading the files

In [ ]:
durcovid_folder_path = '/content/drive/MyDrive/MMT_Project_Data/DurCovidData/hashed_listening_history/'
durcovid_pickle_file = '/content/drive/MyDrive/MMT_Project_Data/durcovid_acoustic_spotipy.pkl'

### Sample code for reading and accessing a pickle file

In [ ]:
#dbfile = open('/content/drive/MyDrive/MMT_Project_Data/PreCovidData/acoustic_11d_final.pkl', 'rb')  
# dbfile = open(durcovid_pickle_file, 'rb')    
# #db = pickle.load(dbfile)
# db2={}
# while 1:
#     try:
#         db2_dict = pickle.load(dbfile)
#         db2 = {**db2, **db2_dict}
#     except EOFError:
#         break

In [ ]:

#stored in a dict, with key = (track_name, artist_name) and value = list of 11 features
# count=0
# for keys in db2:
#     count+=1
#     #print(keys,db[keys])
#     #break
# print(count)

926615


In [ ]:
# pklfile2 = open('/content/drive/MyDrive/MMT_Project_Data/DurCovidData/durcovid_acoustic_file.pkl', 'ab')
# pickle.dump(db2,pklfile2)
# pklfile2.close()

### Function to read user history files and extract acoustic features and write it in a pickle file 

In [ ]:
def read_track(folder_path,pickle_path):
  track_id_set = set()
  dict_all_tracks =db2
  dict_batch_tracks={}
  pkl_file = open(pickle_path, 'wb')
  pickle.dump(dict_all_tracks, pkl_file, protocol=pickle.HIGHEST_PROTOCOL)
  #count=0
  file_count=0
  for file in os.listdir(folder_path):
    print(f'File {file_count} : {file}')
    file_count+=1
    if file.endswith(".csv"): # and file_count>417:        
      user_history = pd.read_csv(folder_path+'/'+file)
      for index,rows in user_history.iterrows():
          #print(index)
          if ((rows['track'],rows['artist']) not in dict_all_tracks):
              track_id = get_track_id(rows['track'],rows['artist'])
              #print((rows['track'],rows['artist']),track_id)
              if (track_id is not None):
                  features = get_features_by_track_ids([track_id])
                  if (features[0] is not None):
                    dict_batch_tracks[(rows['track'],rows['artist'])]=features[0]
                    dict_all_tracks[(rows['track'],rows['artist'])]=features[0]
    pickle.dump(dict_batch_tracks, pkl_file, protocol=pickle.HIGHEST_PROTOCOL)
  return dict_all_tracks

In [ ]:
feature_dict=read_track(durcovid_folder_path,durcovid_pickle_file)

File 0 : 05157feb07f9e780637ffb96ae549365.csv
File 1 : 06bd850533a713bb7800766860dc6e6e.csv
File 2 : 03e7c2869742ac3b8b04a45675ab5b24.csv
File 3 : 0538da9949fdaa2da6e5f18c0a27e146.csv
File 4 : 03c38064fb423b47d5292bffc09785cc.csv
File 5 : 080718a2684221573b174563e37c8f42.csv
File 6 : 05110aecc712b5c4d62efc592507dcce.csv
File 7 : 082ae734bd0f8399771b841e3f19f8a9.csv
File 8 : 056d099110249e4dfc60cfc330f59f76.csv
File 9 : 03a050d649de4bf6e489a064515e2f43.csv
File 10 : 0274a823eec1e677645b20883a5e7897.csv
File 11 : 04e3becfa6b8843aabcfed4e128ffff5.csv
File 12 : 009327643adf158a3693b76e2edde04e.csv
File 13 : 0820db544692f5180aeea5b7233c0f43.csv
File 14 : 026c353028eefcd77442b35ce99832e0.csv
File 15 : 0411187ec1298cca816ce3d255382242.csv
File 16 : 05ed17d0ecc78208ba5d6a3661a9fc95.csv
File 17 : 05a6430c0770691484aae3731acc7d23.csv
File 18 : 049042136e9a0aab809148a6a3016de6.csv
File 19 : 0751cf43a98df47d46d78d2925a608e2.csv
File 20 : 05e2b286c881c6a114eb9e6b229a4cd9.csv
File 21 : 0453c66f7618a

In [ ]:
print(len(feature_dict.keys()))

981557
